<a href="https://colab.research.google.com/github/JoannaLe/cancer-detection/blob/master/models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from keras.applications.resnet50 import ResNet50
from keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from keras.models import Sequential, Model

import numpy as np
import os
import matplotlib.pyplot as plt

In [3]:
# Open Google Cloud Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [9]:
!pwd
!ls

/content
gdrive	sample_data


In [7]:
!ls
x_path = "gdrive/My Drive/cancer-detection/path-zoom-4"
# y_path = "gdrive/My Drive/cancer-detection/mask-zoom-4"
os.chdir(x_path)
for file in os.listdir(x_path):
  path_file = os.path.join(x_path, file)
  f = open(path_file, "r")
  arr = f.read()
  print(str(arr))


gdrive	sample_data


FileNotFoundError: ignored

## Model 1
4 resnet models with a concatenation layer

In [0]:
img_shape = (300,300,3)

zoom1 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
zoom2 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
zoom3 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
zoom4 = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)

# concatenation layer
concat = Average()([zoom1.output, zoom2.output, zoom3.output, zoom4.output])

full_model = Model([zoom1.input, zoom2.input, zoom3.input, zoom4.input], concat)


In [0]:
# to train, call
EPOCHS = 20
BATCH_SIZE = 32

full_model.fit([X_train1, X_train2, X_train3, X_train4],
               [Y_train1, Y_train2, Y_train3, Y_train4],
                epochs=EPOCHS,
                batch_size=BATCH_SIZE)

score = model.evaluate([X_test1, X_test2, X_test3, X_test4],
                       [Y_test1, Y_test2, Y_test3, Y_test4],
                       batch_size=BATCH_SIZE)

# Model 2
Transfer Learning using weights from Resnet \\
Extract features from Resnet weights and training/test data

In [0]:
img_shape = (300,300,3)

base = ResNet50(include_top=True, weights='imagenet', input_shape=img_shape)
